In [1]:
import xmltodict
import json

print(json.dumps(xmltodict.parse("""
  <mydocument has="an attribute">
    <and>
      <many>elements</many>
      <many>more elements</many>
    </and>
    <plus a="complex">
      element as well
    </plus>
  </mydocument>
  """), indent=4))


{
    "mydocument": {
        "@has": "an attribute",
        "and": {
            "many": [
                "elements",
                "more elements"
            ]
        },
        "plus": {
            "@a": "complex",
            "#text": "element as well"
        }
    }
}


In [2]:
import xmltodict
import json
import pandas as pd

# with open('consol.xml', 'r') as f:
with open('consolidated.xml', 'r') as f:
    data = f.read()

# print(data[:300])

consol_dict = xmltodict.parse(data)

consol_json = json.dumps(consol_dict, indent=2)

print(consol_json[:6000])
# print(consol_json)


# indivs = consol['CONSOLIDATED_LIST']['INDIVIDUALS']['INDIVIDUAL']
# print(type(indivs))
# for item in indivs:
#     print(item)

# # consol_json = json.dumps(indivs, indent=2)

# # df = pd.read_json(consol_json)
# df = pd.read_json(json.dumsp(indivs))


{
  "CONSOLIDATED_LIST": {
    "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "@xsi:noNamespaceSchemaLocation": "https://www.un.org/sc/resources/sc-sanctions.xsd",
    "@dateGenerated": "2022-10-21T20:40:12.605-04:00",
    "INDIVIDUALS": {
      "INDIVIDUAL": [
        {
          "DATAID": "6908555",
          "VERSIONNUM": "1",
          "FIRST_NAME": "RI",
          "SECOND_NAME": "WON HO",
          "THIRD_NAME": null,
          "UN_LIST_TYPE": "DPRK",
          "REFERENCE_NUMBER": "KPi.033",
          "LISTED_ON": "2016-11-30",
          "COMMENTS1": "Ri Won Ho is a DPRK Ministry of State Security Official stationed in Syria supporting KOMID.",
          "DESIGNATION": {
            "VALUE": "DPRK Ministry of State Security Official"
          },
          "NATIONALITY": {
            "VALUE": "Democratic People's Republic of Korea"
          },
          "LIST_TYPE": {
            "VALUE": "UN List"
          },
          "LAST_DAY_UPDATED": {
            "VALUE":

In [3]:
indivs = consol_dict['CONSOLIDATED_LIST']['INDIVIDUALS']['INDIVIDUAL']
print(type(indivs))

df_nested_list = pd.json_normalize(indivs,
                                    meta=[
                                          ['INDIVIDUAL_ADDRESS', 'COUNTRY', 'STATE_PROVINCE','ZIP_CODE', 'NOTE'],
                                          ['INDIVIDUAL_DOCUMENT', 'TYPE_OF_DOCUMENT', 'NUMBER', 'ISSUING_COUNTRY', 'NOTE'],
                                    ['INDIVIDUAL_DATE_OF_BIRTH', 'TYPE_OF_DATE', 'DATE'],
                                    ['INDIVIDUAL_ALIAS', 'QUALITY', 'ALIAS_NAME']
                                          ]                                   
                                   )  #, record_path =['INDIVIDUAL'])
df_nested_list

<class 'list'>


,DATAID,VERSIONNUM,FIRST_NAME,SECOND_NAME,THIRD_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,INDIVIDUAL_PLACE_OF_BIRTH,...,INDIVIDUAL_ADDRESS.STATE_PROVINCE,INDIVIDUAL_ADDRESS.ZIP_CODE,INDIVIDUAL_ALIAS.DATE_OF_BIRTH,INDIVIDUAL_ALIAS.CITY_OF_BIRTH,INDIVIDUAL_ALIAS.COUNTRY_OF_BIRTH,INDIVIDUAL_DOCUMENT.NOTE,INDIVIDUAL_DATE_OF_BIRTH.NOTE,INDIVIDUAL_DOCUMENT.CITY_OF_ISSUE,SUBMITTED_BY,INDIVIDUAL_PLACE_OF_BIRTH.NOTE
0,6908555,1,RI,WON HO,None,DPRK,KPi.033,2016-11-30,Ri Won Ho is a DPRK Ministry of State Security...,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6908570,1,CHANG,CHANG HA,None,DPRK,KPi.037,2016-11-30,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6908571,1,CHO,CHUN RYONG,None,DPRK,KPi.038,2016-11-30,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6908858,1,EMRAAN,ALI,None,Al-Qaida,QDi.430,2021-11-23,Senior member of Islamic State in Iraq and the...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6908565,1,JO,YONG CHOL,None,DPRK,KPi.034,2016-11-30,Jo Yong Chol is a DPRK Ministry of State Secur...,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,6908704,1,Ahmad,Oumar,Imhamad,Libya,LYi.023,2018-06-07,Listed pursuant to paragraphs 15 and 17 of res...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"possibly Sabratha, Talil neighbourhood"
699,6908707,1,Abd,Al-Rahman,al-Milad,Libya,LYi.026,2018-06-07,Listed pursuant to paragraphs 15 and 17 of res...,NaN,...,NaN,NaN,NaN,NaN,NaN,Date of expiration: 7 May 2022,NaN,NaN,NaN,NaN
700,6908841,1,Amir,Muhammad Sa’id,Abdal-Rahman,Al-Qaida,QDi.426,2020-05-21,Leader of Islamic State in Iraq and the\n ...,"[{'STATE_PROVINCE': 'Tall’Afar', 'COUNTRY': 'I...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,2975510,1,FAIZULLAH,KHAN,NOORZAI,Taliban,TAi.153,2011-10-04,"Prominent Taliban financier. As of mid-2009, s...","[{'CITY': 'Lowy Kariz, Spin Boldak District', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# df_select = df_nested_list[['FIRST_NAME','SECOND_NAME', 'THIRD_NAME', 'REFERENCE_NUMBER','COMMENTS1']]
# # df_nested_list.info()
# df_select

,FIRST_NAME,SECOND_NAME,THIRD_NAME,REFERENCE_NUMBER,COMMENTS1
0,RI,WON HO,None,KPi.033,Ri Won Ho is a DPRK Ministry of State Security...
1,CHANG,CHANG HA,None,KPi.037,None
2,CHO,CHUN RYONG,None,KPi.038,None
3,EMRAAN,ALI,None,QDi.430,Senior member of Islamic State in Iraq and the...
4,JO,YONG CHOL,None,KPi.034,Jo Yong Chol is a DPRK Ministry of State Secur...
...,...,...,...,...,...
698,Ahmad,Oumar,Imhamad,LYi.023,Listed pursuant to paragraphs 15 and 17 of res...
699,Abd,Al-Rahman,al-Milad,LYi.026,Listed pursuant to paragraphs 15 and 17 of res...
700,Amir,Muhammad Sa’id,Abdal-Rahman,QDi.426,Leader of Islamic State in Iraq and the\n ...
701,FAIZULLAH,KHAN,NOORZAI,TAi.153,"Prominent Taliban financier. As of mid-2009, s..."


In [72]:
# pd.set_option('display.expand_frame_repr', True)
# pd.set_option('max_colwidth', -1)
import re
rstring = "(\([a-z]{3,4}.\d{3,4}\))"
df_nested_list['CONNECTIONS'] = df_nested_list.loc[:,'COMMENTS1'].str.findall(r'\([a-z]{3,4}.\d{3,4}\)', flags=re.IGNORECASE)
df_nested_list['CONN_COUNT'] = df_nested_list['CONNECTIONS'].str.len()
# df_nested_list.info()

print(df_nested_list.loc[(df_nested_list['CONNECTIONS'].notnull()) & (df_nested_list['CONN_COUNT']>1)],['CONNECTIONS','CONN_COUNT'])

# display(dataFrame.loc[(dataFrame['Salary']>=100000) & (dataFrame['Age']< 40) & (dataFrame['JOB'].str.startswith('D')),
# print(df_nested_list[[(df_nested_list.CONNECTIONS.notnull()) and ('CONN_COUNT'>1)]][['CONNECTIONS', 'COMMENTS1']])  # [['CONNECTIONS' == None]]
# https://regex101.com/r/ejCzvg/1
# [itm for itm in S.str.extractall('(.*)') if (itm is not None and len(str(itm))>0)]
# [itm[0] for itm in S.str.findall('(.*)') if (itm is not None and len(itm)>0)]
# [itm[0] for itm in S.str.findall('^[Ff].*') if (itm is not None and len(itm)>0)]
# find code in cyber snorkel that counts everything

      DATAID VERSIONNUM    FIRST_NAME  SECOND_NAME           THIRD_NAME  \
14   6908440  1          ASHRAF        MUHAMMAD     YUSUF                 
16   2975591  1          ABDUL AZIZ    ABBASIN      NaN                   
26   2989674  1          ABDUL SATAR   ABDUL MANAN  NaN                   
31   2972739  1          MUHAMMAD      JIBRIL       ABDUL RAHMAN          
35   2983651  1          ABDUR REHMAN  NaN          NaN                   
..       ... ..                   ...  ...          ...                   
647  111639   1          IBRAHIM       ALI          ABU BAKR              
654  2813362  1          ANGELO        RAMIREZ      TRINIDAD              
670  111039   1          UBAIDULLAH    AKHUND       YAR MOHAMMAD AKHUND   
673  2830354  1          ADEM          YILMAZ       NaN                   
689  3000769  1          ABDUL RAUF    ZAKIR        NaN                   

    UN_LIST_TYPE REFERENCE_NUMBER   LISTED_ON  \
14   Al-Qaida     QDi.343          2015-01-23   
1

In [8]:
S=pd.Series(['Finland','Colombia','Florida','Japan','Puerto Rico','Russia','france'])
[itm[0] for itm in S.str.findall('^[Ff].*') if len(itm)>0]


['Finland', 'Florida', 'france']

In [6]:
# Add Preprocessors
# feature_eng_ip_pair_cnt = GroupByCountPreprocessor(func_kwargs={"resulting_column_name":"ip_pair_count",
#         "groupby_list":["id_orig_h","id_resp_h"],
#         "count_key":"uid"})
# self.output_obj[kwargs["resulting_column_name"]] = \
#     input_obj.groupby(
#         kwargs["groupby_list"])[kwargs["count_key"]].transform('count')